In [1]:
from SVM_implementation import build_models, runner
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
transformer_config = {
    "data_path": "../project_data/t4sa_data.csv",  # path to the csv with embeddings
}

w2v_config = {
    "data_path"   : "../project_data/t4sa_data_w2v_sg_test.npz"
}

In [3]:
# run transformer
runner(transformer_config)

Parsing embeddings...
Epoch0: 984/3000(32.80%)
Epoch1: 920/3000(30.67%)
Epoch2: 1094/3000(36.47%)
Epoch3: 1565/3000(52.17%)
Epoch4: 1701/3000(56.70%)
Epoch5: 1771/3000(59.03%)
Epoch6: 1764/3000(58.80%)
Epoch7: 1870/3000(62.33%)
Epoch8: 1985/3000(66.17%)
Epoch9: 2061/3000(68.70%)
Epoch10: 2027/3000(67.57%)
Epoch11: 2081/3000(69.37%)
Epoch12: 2109/3000(70.30%)
Epoch13: 2116/3000(70.53%)
Epoch14: 2140/3000(71.33%)
Epoch15: 2150/3000(71.67%)
Epoch16: 2177/3000(72.57%)
Epoch17: 2213/3000(73.77%)
Epoch18: 2221/3000(74.03%)
Epoch19: 2243/3000(74.77%)
Epoch20: 2258/3000(75.27%)
Epoch21: 2284/3000(76.13%)
Epoch22: 2304/3000(76.80%)
Epoch23: 2310/3000(77.00%)
Epoch24: 2325/3000(77.50%)
Epoch25: 2346/3000(78.20%)
Epoch26: 2361/3000(78.70%)
Epoch27: 2368/3000(78.93%)
Epoch28: 2391/3000(79.70%)
Epoch29: 2409/3000(80.30%)
Epoch30: 2423/3000(80.77%)
Epoch31: 2437/3000(81.23%)
Epoch32: 2444/3000(81.47%)
Epoch33: 2448/3000(81.60%)
Epoch34: 2442/3000(81.40%)
Epoch35: 2453/3000(81.77%)
Epoch36: 2459/3000

In [4]:
w2v_arr = np.load(w2v_config["data_path"], allow_pickle=True)

In [5]:
w2v_arr[w2v_arr.files[0]].shape

(50000, 5)

In [6]:
data = pd.DataFrame(
    w2v_arr[w2v_arr.files[0]],
    columns = ["text", "class", "tokenized_text", "embeddings", "embeddings_flat"]
    )

In [10]:
embedding_list =  [ np.array(l) for l in data["embeddings_flat"]]
max_len = max(l.shape[0] for l in embedding_list)
data["embeddings_flat"] = [np.pad(l, (0, max_len - l.shape[0])) for l in embedding_list]

In [8]:
def w2v_runner():
    custom_mlp, sk_svm = build_models()
    # data = 
    # data = pd.read_csv(configs["data_path"])


    print("Parsing embeddings...")
    if isinstance(data['embeddings'].iloc[0], str):
        data['embeddings_flat'] = data['embeddings_flat'].str.strip('[]').str.split(',').apply(
            lambda x: np.array([float(i) for i in x])
        )

    X, y = np.stack(data['embeddings_flat'].values), data['class']
    # encode the class labels
    y_unique = {label: idx for idx, label in enumerate(sorted(y.unique()))}
    y = y.map(y_unique).values

    # train-test split
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
    )

    # train the custom MLP
    custom_mlp.fit(X_train, y_train, X_val, y_val)
    acc_mlp = custom_mlp.evaluate((X_test, y_test))
    print(f"Custom MLP Test Accuracy: {acc_mlp:.4f}")

    # train the scikit-learn SVM
    sk_svm.fit(X_train, y_train)
    acc_svm = sk_svm.score(X_test, y_test)
    print(f"Scikit-learn SVM Test Accuracy on Word2vec embedded text: {acc_svm:.4f}")

In [9]:
w2v_runner()

Parsing embeddings...
Epoch0: 6352/10000(63.52%)
Epoch1: 6368/10000(63.68%)
Epoch2: 6356/10000(63.56%)
Epoch3: 6431/10000(64.31%)
Epoch4: 6642/10000(66.42%)
Epoch5: 6814/10000(68.14%)
Epoch6: 7015/10000(70.15%)
Epoch7: 7097/10000(70.97%)
Epoch8: 7250/10000(72.50%)
Epoch9: 7272/10000(72.72%)
Epoch10: 7388/10000(73.88%)
Epoch11: 7445/10000(74.45%)
Epoch12: 7467/10000(74.67%)
Epoch13: 7547/10000(75.47%)
Epoch14: 7570/10000(75.70%)
Epoch15: 7653/10000(76.53%)
Epoch16: 7655/10000(76.55%)
Epoch17: 7692/10000(76.92%)
Epoch18: 7705/10000(77.05%)
Epoch19: 7717/10000(77.17%)
Epoch20: 7747/10000(77.47%)
Epoch21: 7745/10000(77.45%)
Epoch22: 7788/10000(77.88%)
Epoch23: 7820/10000(78.20%)
Epoch24: 7829/10000(78.29%)
Epoch25: 7881/10000(78.81%)
Epoch26: 7886/10000(78.86%)
Epoch27: 7890/10000(78.90%)
Epoch28: 7887/10000(78.87%)
Epoch29: 7866/10000(78.66%)
Epoch30: 7908/10000(79.08%)
Epoch31: 7927/10000(79.27%)
Epoch32: 7938/10000(79.38%)
Epoch33: 7929/10000(79.29%)
Epoch34: 7954/10000(79.54%)
Epoch35: